In [1]:
import sys
import os

sys.path.insert(0, os.path.dirname(os.getcwd()))

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, rand, split, explode, regexp_replace

from schemas.dataframes import get_episode_df, get_basics_df, get_akas_df, get_crew_df, get_principals_df, get_ratings_df, get_name_df

d:\study\circus\HDFS\project\github\big-data-project


In [2]:
spark_session = (SparkSession.builder
                             .master('local')
                             .appName('test app')
                             .config(conf=SparkConf())
                             .getOrCreate())

In [8]:
title_episode = get_episode_df(spark_session)

title_basic = get_basics_df(spark_session)

title_akas = get_akas_df(spark_session)

rating = get_ratings_df(spark_session)

In [4]:
get_episode_df(spark_session).columns

['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber']

Query 1(Кількість )

In [5]:
merged_data = title_episode.join(title_basic, title_episode['parentTconst'] == title_basic['tconst'], 'inner') \
                           .orderBy(rand()) \
                           .select("primaryTitle", "originalTitle", col("startYear").alias("Year"), "seasonNumber", "episodeNumber" , "genres") \
                           .na.drop() \
                           .limit(10)

merged_data.show()

+--------------------+--------------------+----+------------+-------------+--------------------+
|        primaryTitle|       originalTitle|Year|seasonNumber|episodeNumber|              genres|
+--------------------+--------------------+----+------------+-------------+--------------------+
|    Maalaala Mo Kaya|    Maalaala mo kaya|1991|           1|          903|Biography,Drama,R...|
|The Young and the...|The Young and the...|1973|           1|         2749|       Drama,Romance|
|           Marinella|           Marinella|1999|           1|          379|       Drama,Romance|
|Tengo algo que En...|Tengo algo que En...|2017|           3|            7|  Comedy,Documentary|
|The Really Wild Show|The Really Wild Show|1986|          13|           11|  Documentary,Family|
| Me Gusta: Með Gústa| Me Gusta: Með Gústa|2013|           1|            2|              Comedy|
|        What A Charm|        Yahsi Cazibe|2010|           2|           21|              Comedy|
|               La ex|        

Query 2 (найбільш тривалі серіали\фільми за жанрами)

In [27]:
start_date = "2020"
end_date = "2023"

# filter: Групування фільмів за жанром
filtered_movies = title_basic.filter((col("startYear") >= start_date) & (col("startYear") < end_date))

# Розділення поля "genres" на різні стовпці
split_genres = (filtered_movies.withColumn("genres_array", split(regexp_replace(col("genres"), "\\\\N", ""), ",")))

# Використання explode для розгортання масиву жанрів
exploded_genres = split_genres.select("tconst", explode("genres_array").alias("genre"), "runtimeMinutes")

# Group By: Групування фільмів за жанром
grouped_movies = exploded_genres.groupBy("genre")

# Window Function: Обчислення середнього рейтингу для кожного жанру
average_rating_per_genre = grouped_movies.agg({"runtimeMinutes": "avg"}).limit(10)

sorted_result = average_rating_per_genre.orderBy(col("avg(runtimeMinutes)").desc())

# Виведення результату
sorted_result.show()

+-----------+-------------------+
|      genre|avg(runtimeMinutes)|
+-----------+-------------------+
|        War| 57.662944162436546|
| Reality-TV|  54.77462393026099|
|   Thriller| 51.212988442487614|
|      Crime|  49.19968429360694|
|    Romance|  47.95665236051502|
|    Fantasy|  46.59091938405797|
|Documentary| 44.673557804663204|
|  Adventure| 44.585649044043656|
|      Drama| 43.013921909560814|
|     Family|  34.47700718190588|
+-----------+-------------------+



Query 3 - продубльовані фільми українською до 1950 року

In [13]:
ukrainian_movies = (
    title_akas
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'], 'inner')
    .filter((col("language") == "uk") & (col("startYear") < 1950))
    .orderBy(rand())
    .select("primaryTitle", "originalTitle", "language", "startYear", "genres")
    .limit(10)
)

ukrainian_movies.show()

+--------------------+--------------------+--------+---------+--------------------+
|        primaryTitle|       originalTitle|language|startYear|              genres|
+--------------------+--------------------+--------+---------+--------------------+
|                Alim|                Alim|      uk|     1926|               Drama|
|   The House of Hate|   The House of Hate|      uk|     1918|        Action,Drama|
|Za monastyrskoyu ...|Za monastyrskoyu ...|      uk|     1928|               Drama|
|             Hamburg|             Gamburg|      uk|     1926|               Drama|
|  The Night Coachman|   Nochnoj izvozchik|      uk|     1929|               Drama|
|Za monastyrskoyu ...|Za monastyrskoyu ...|      uk|     1928|               Drama|
|             Hamburg|             Gamburg|      uk|     1926|               Drama|
|         V sugrobakh|         V sugrobakh|      uk|     1929|Adventure,Drama,H...|
|       Yego kar'yera|       Yego kar'yera|      uk|     1928|              

Query 4

In [10]:
ukrainian_movies = (
    title_akas
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'], 'inner')
    .join(rating, title_basic['tconst'] == rating['tconst'])
    .filter((col("language") == "uk") & (col("startYear") > 2000))
    .select("primaryTitle", "originalTitle", "language", "startYear", "genres", "averageRating")
    .orderBy(col("averageRating").desc())
    .limit(10)
)

ukrainian_movies.show()

+--------------------+--------------------+--------+---------+--------------------+-------------+
|        primaryTitle|       originalTitle|language|startYear|              genres|averageRating|
+--------------------+--------------------+--------+---------+--------------------+-------------+
|Mission: Impossib...|Mission: Impossib...|      uk|     2018|Action,Adventure,...|          7.7|
|   War of the Worlds|   War of the Worlds|      uk|     2005|Action,Adventure,...|          6.5|
|            I, Robot|            I, Robot|      uk|     2019|   Documentary,Short|          6.3|
|   A Wrinkle in Time|   A Wrinkle in Time|      uk|     2018|Adventure,Drama,F...|          4.3|
+--------------------+--------------------+--------+---------+--------------------+-------------+

